In [193]:
import numpy as np
from scipy.stats import norm
from scipy import optimize

# Delta hedging

Suppose we sell an option on 1000 shares of stock ABC where 
\begin{eqnarray}
S_0 = \text{spot price} \\
X = \text{strike price} \\
r = \text{interest rate} \\
sigma = \text{volatility} \\
T = \text{time to expiration}
\end{eqnarray}

In [194]:
def delta_hedger(S_0, X, r, sigma, T):
    d_1 = (np.log(S_0/X) + (r + ((sigma**2)/2))*T)/(sigma*np.sqrt(T))
    return norm.cdf(d_1)

In [195]:
(delta_hedger(50, 40, 0.05, 0.30, 52/52)*1000), delta_hedger(51.5, 40, 0.05, 0.30, 51/52)*1000, delta_hedger(49, 40, 0.05, 0.30, 50/52)*1000

(855.5365179482682, 877.8216978021413, 841.4375070514154)

There is a drawback with this approach as we can see. When the price increases from 50 to 51.5, the number of stocks required for hedging goes up. Moreover, when the price reduces to 49, we need to reduce the number of stocks accordingly. 
# bad-trade-deal
<div align = 'center'>
<b> We are buying up and selling low. We are always going the wrong way on stock trades.<b>
</div>

# Implied volatility

In [196]:
def implied_vol_cal(V, S_0, X, r, T):
    def black_schole_price(x):
        d_1 = (np.log(S_0/X) + (r + ((x**2)/2))*T)/(x*np.sqrt(T))
        d_2 = d_1 - (x*np.sqrt(T))
    
        return (S_0*(norm.cdf(d_1)) - X*(np.exp((-r)*T)*norm.cdf(d_2)) - V)
        
    return optimize.brentq(black_schole_price, 10**(-6), 1)

In [197]:
implied_vol_cal(3.8, 40, 40, 0.05, 3/12), implied_vol_cal(2, 45, 60, 0.047, 3/12) 

(0.448146521468883, 0.6730068343895313)

# Updating the call price using $\Delta, \Gamma$, $\Theta$ and using the approximation of $dC$

We have
\begin{eqnarray}
dC &\approx& \frac{\partial C}{\partial t}dt + \frac{\partial C}{\partial S}dS + \frac{\sigma^2}{2}S^2\frac{\partial^2 C}{\partial S^2} \\
&\approx& \Theta dt + \Delta dS + \Gamma \frac{1}{2}(dS)^2
\end{eqnarray}
where 
\begin{eqnarray}
\Delta &=& \frac{\partial C}{\partial S} = N(d_1) \\
\Gamma &=& \frac{\partial^2 C}{\partial S^2} = \frac{1}{\sigma S \sqrt{2\pi T}}e^{-d_1^2/2} \\
\Theta &=& \frac{\partial C}{\partial t} = -re^{-rT}XN(d_2) - \frac{1}{2}\sigma^2 S^2 \Gamma.
\end{eqnarray}
Therefore $$C_{new} = C_{old} + dC$$
where $C_{old} = $ black_Schole_pricer$(S_0, X, sigma, r, T)$

To address the <span style="color:red"><i>bad-trade-deal</i></span> , we revise our hedging strategies as follows.:
<div align = 'centre'>
    <b>Instead of buying/selling $\Delta$ quantity of stocks against a call, we buy/sell $\frac{1}{\Delta}$ quantity of call against a stock<b>
</div>

        # Advantage with the above stratigy is the following:
- when stock price increases, $d_1$ goes up and inturn $\Delta$ and call price increases. However, $1/\Delta$ reduces, which means we need to reduce the number of call.
- OTOH when stock prices reduces, $d_1$ goes down and hence $\Delta$ and call prices reduces. But $1/\Delta$ increases, therefore we need to increase the number of call.

In [204]:
def delta_gamma_theta_cal(S_0, X, sigma, r, T):
    d_1 = (np.log(S_0/X) + ((r + (sigma**2)/2)*T))/(sigma*np.sqrt(T))
    d_2 = d_1 - sigma*np.sqrt(T)
    delta = norm.cdf(d_1)
    gamma = np.exp(-((d_1)**2)/2)/(sigma*(S_0*np.sqrt(2*(np.pi*T))))
    theta = -r*np.exp(-r*T)*(X*norm.cdf(d_2)) - 0.5*sigma**2*((S_0**2)*gamma)
    return delta, gamma, theta

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [205]:
delta_gamma_theta_cal(50, 55, 0.30, 0.05, 6/12), delta_gamma_theta_cal(60, 60, 0.33, 0.052, 3/12)

((0.41084208775394915, 0.03666938495373566, -5.012733682039494),
 (0.5640666652002937, 0.03977645278816542, -9.332270737142196))

In [206]:
def black_Schole_pricer(S_0, X, sigma, r, T):
    d_1 = (np.log(S_0/X) + ((r + (sigma**2)/2)*T))/(sigma*(np.sqrt(T)))
    d_2 = d_1-(sigma*(np.sqrt(T)))
    
    N_d_1 = norm.cdf(d_1)
    N_d_2 = norm.cdf(d_2)
    
    C = (N_d_1*S_0) - N_d_2*(X*np.exp(-(r*T)))
    exp_payoff = N_d_2*(X*np.exp(-(r*T)))
    
    return C

In [207]:
S_0 = 43
X = 40
sigma = 0.1414
r = 0.05
T = 12/12
dt = 3/52
S_new = 44
dS = S_new - S_0
delta = delta_gamma_theta_cal(S_0, X, sigma, r, T)[0]
gamma = delta_gamma_theta_cal(S_0, X, sigma, r, T)[1]
theta = delta_gamma_theta_cal(S_0, X, sigma, r, T)[2]

C_old = black_Schole_pricer(S_0, X, sigma, r, T)

dC = (theta*dt) + ((delta*dS) + (0.5*gamma*(dS**2)))
C_new = C_old + dC

In [208]:
print(C_new), black_Schole_pricer(44, 40, 0.1414, 0.05, 12/12 - 3/52)

6.277014912466308


(None, 6.276216041646972)

Note that the two values C_new and black_Schole_pricer() are same which they should be.